In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split

caltech_dir = "/content/drive/My Drive/Colab Notebooks/dataset/all"
categories = ["AFRICAN FIREFINCH", "ALBATROSS", "ALEXANDRINE PARAKEET", "ALOUATTAPALLIATA", "AMERICAN AVOCET", "AMERICAN BITTERN", "AOTUSNIGRICEPS", "ASIAN CRESTED IBIS", "BLACK SWAN", "BLACK THROATED WARBLER", "BLACK VULTURE", "BLACK-CAPPED CHICKADEE", "BLACK-THROATED SPARROW", "BLUE HERON", "BROWN THRASHER", "CACAJAOCALVUS", "CEBUELLAPYGMEA", "CEBUSCAPUCINUS", "COCK OF THE ROCK", "COMMON GRACKLE", "COMMON STARLING", "COUCHS KINGBIRD", "ERYTHROCEBUSPATAS", "FLAMINGO", "FRIGATE", "GAMBELS QUAIL", "GOLD WING WARBLER", "MACACAFUSCATA", "MICOARGENTATUS", "PERSIAN CAT", "RUSSIAN BLUE", "SAIMIRISCIUREUS", "SPHYNX CAT", "TRACHYPITHECUSJOHNII"]
nb_classes = len(categories)

image_w = 32
image_h = 32

pixels = image_h * image_w * 3

X = []
y = []

for idx, cat in enumerate(categories):
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    image_dir = caltech_dir + "/" + cat
    files = glob.glob(image_dir+"/*.*")
    img_num = 0
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        X.append(data)
        y.append(label)

        if i % 3000 == 0:
            print(cat, " : ", f)
        img_num += 1
        if i >= 6000: break
    print(cat, " 이미지 개수 : ", img_num)
print("전체 이미지 개수 : ", len(X))

X = np.array(X)
y = np.array(y)
#1 0 0 0 이면 airplanes
#0 1 0 0 이면 buddha 이런식


X_train, X_test, y_train, y_test = train_test_split(X, y)
xy = (X_train, X_test, y_train, y_test)
np.save("/content/drive/My Drive/Colab Notebooks/numpy_data/multi_image_data2.npy", xy)

print("ok", len(y))


AFRICAN FIREFINCH  :  /content/drive/My Drive/Colab Notebooks/dataset/all/AFRICAN FIREFINCH/AFRICAN FIREFINCH_67_8839311.jpeg
AFRICAN FIREFINCH  :  /content/drive/My Drive/Colab Notebooks/dataset/all/AFRICAN FIREFINCH/AFRICAN FIREFINCH_42_8581265.jpeg
AFRICAN FIREFINCH  :  /content/drive/My Drive/Colab Notebooks/dataset/all/AFRICAN FIREFINCH/AFRICAN FIREFINCH_6_4507445.jpeg
AFRICAN FIREFINCH  이미지 개수 :  6001
전체 이미지 개수 :  6001
ALBATROSS  :  /content/drive/My Drive/Colab Notebooks/dataset/all/ALBATROSS/ALBATROSS_117_9734984.jpeg
ALBATROSS  :  /content/drive/My Drive/Colab Notebooks/dataset/all/ALBATROSS/ALBATROSS_15_2366986.jpeg
ALBATROSS  :  /content/drive/My Drive/Colab Notebooks/dataset/all/ALBATROSS/ALBATROSS_37_2763901.jpeg
ALBATROSS  이미지 개수 :  6001
전체 이미지 개수 :  12002
ALEXANDRINE PARAKEET  이미지 개수 :  0
전체 이미지 개수 :  12002
ALOUATTAPALLIATA  :  /content/drive/My Drive/Colab Notebooks/dataset/all/ALOUATTAPALLIATA/alouattapalliata_4_4677130.jpeg
ALOUATTAPALLIATA  :  /content/drive/My Drive

In [ ]:
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow as tf
import keras.backend as K

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

X_train, X_test, y_train, y_test = np.load('/content/drive/My Drive/Colab Notebooks/numpy_data/multi_image_data.npy', allow_pickle=True)
print(X_train.shape)
print(X_train.shape[0])

(23487, 32, 32, 3)
23487


In [ ]:
categories = ["AFRICAN FIREFINCH", "ALBATROSS", "ALEXANDRINE PARAKEET", "ALOUATTAPALLIATA", "AMERICAN AVOCET", "AMERICAN BITTERN", "AOTUSNIGRICEPS", "ASIAN CRESTED IBIS", "BLACK SWAN", "BLACK THROATED WARBLER", "BLACK VULTURE", "BLACK-CAPPED CHICKADEE", "BLACK-THROATED SPARROW", "BLUE HERON", "BROWN THRASHER", "CACAJAOCALVUS", "CEBUELLAPYGMEA", "CEBUSCAPUCINUS", "COCK OF THE ROCK", "COMMON GRACKLE", "COMMON STARLING", "COUCHS KINGBIRD", "ERYTHROCEBUSPATAS", "FLAMINGO", "FRIGATE", "GAMBELS QUAIL", "GOLD WING WARBLER", "MACACAFUSCATA", "MICOARGENTATUS", "PERSIAN CAT", "RUSSIAN BLUE", "SAIMIRISCIUREUS", "SPHYNX CAT", "TRACHYPITHECUSJOHNII" ]
nb_classes = len(categories)

#일반화
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255

In [ ]:
#with K.tf.device('/device:GPU:0'):
model = Sequential()
model.add(Conv2D(32, (3,3), padding="same", activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
    
model.add(Conv2D(64, (3,3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_dir = './model'
    
if not os.path.exists(model_dir):
  os.mkdir(model_dir)
  
  model_path = model_dir + '/multi_img_classification.model'
  checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=1, save_best_only=True)
  early_stopping = EarlyStopping(monitor='val_loss', patience=6)
    
    

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 8, 8, 128)        

In [ ]:
print("정확도 : %.4f" % (model.evaluate(X_test, y_test)[1]))

In [ ]:
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))

plt.plot(x_len, y_vloss, marker='.', c='red', label='val_set_loss')
plt.plot(x_len, y_loss, marker='.', c='blue', label='train_set_loss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()
plt.show()

In [ ]:
y_vloss = history.history['val_acc']
y_loss = history.history['acc']

x_len = np.arange(len(y_loss))

plt.plot(x_len, y_vloss, marker='.', c='red', label='val_set_loss')
plt.plot(x_len, y_loss, marker='.', c='blue', label='train_set_loss')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('acc')
plt.grid()
plt.show()

In [ ]:
from PIL import Image
import os, glob, numpy as np
from keras.models import load_model


caltech_dir = "/content/drive/My Drive/Colab Notebooks/MIX/bird"
image_w = 32
image_h = 32

pixels = image_h * image_w * 3

X = []
filenames = []
files = glob.glob(caltech_dir+"/*.*")
for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    filenames.append(f)
    X.append(data)

X = np.array(X)
model = load_model('./model/multi_img_classification.model')

prediction = model.predict(X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
cnt = 0

#이 비교는 그냥 파일들이 있으면 해당 파일과 비교. 카테고리와 함께 비교해서 진행하는 것은 _4 파일.
for i in prediction:
    pre_ans = i.argmax()  # 예측 레이블
    print(i)
    print(pre_ans)
    pre_ans_str = ''
    if pre_ans == 0: pre_ans_str = "AFRICAN FIREFINCH"
    elif pre_ans == 1: pre_ans_str = "ALBATROSS"
    elif pre_ans == 2: pre_ans_str = "ALEXANDRINE PARAKEET"
    elif pre_ans == 3: pre_ans_str = "ALOUATTAPALLIATA"
    elif pre_ans == 4: pre_ans_str = "AMERICAN AVOCET"
    elif pre_ans == 5: pre_ans_str = "AMERICAN BITTERN"
    elif pre_ans == 6: pre_ans_str = "AOTUSNIGRICEPS"
    elif pre_ans == 7: pre_ans_str = "ASIAN CRESTED IBIS"
    elif pre_ans == 8: pre_ans_str = "BLACK SWAN"
    elif pre_ans == 9: pre_ans_str = "BLACK THROATED WARBLER"
    elif pre_ans == 10: pre_ans_str = "BLACK VULTURE"
    elif pre_ans == 11: pre_ans_str = "BLACK-CAPPED CHICKADEE"
    elif pre_ans == 12: pre_ans_str = "BLACK-THROATED SPARROW"
    elif pre_ans == 13: pre_ans_str = "BLUE HERON"
    elif pre_ans == 14: pre_ans_str = "BROWN THRASHER"
    elif pre_ans == 15: pre_ans_str = "CACAJAOCALVUS"
    elif pre_ans == 16: pre_ans_str = "CEBUELLAPYGMEA"
    elif pre_ans == 17: pre_ans_str = "CEBUSCAPUCINUS"
    elif pre_ans == 18: pre_ans_str = "COCK OF THE ROCK"
    elif pre_ans == 19: pre_ans_str = "COMMON GRACKLE"
    elif pre_ans == 20: pre_ans_str = "COMMON STARLING"
    elif pre_ans == 21: pre_ans_str = "COUCHS KINGBIRD"
    elif pre_ans == 22: pre_ans_str = "ERYTHROCEBUSPATAS"
    elif pre_ans == 23: pre_ans_str = "FLAMINGO"
    elif pre_ans == 24: pre_ans_str = "FRIGATE"
    elif pre_ans == 25: pre_ans_str = "GAMBELS QUAIL"
    elif pre_ans == 26: pre_ans_str = "GOLD WING WARBLER"
    elif pre_ans == 27: pre_ans_str = "MACACAFUSCATA"
    elif pre_ans == 28: pre_ans_str = "MICOARGENTATUS"
    elif pre_ans == 29: pre_ans_str = "PERSIAN CAT"
    elif pre_ans == 30: pre_ans_str = "RUSSIAN BLUE"
    elif pre_ans == 31: pre_ans_str = "SAIMIRISCIUREUS"
    elif pre_ans == 32: pre_ans_str = "SPHYNX CAT"
    else: pre_ans_str = "TRACHYPITHECUSJOHNII"
    if i[0] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[1] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[2] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[3] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[4] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[5] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[6] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[7] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[8] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[9] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[10] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[11] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[12] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[13] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[14] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[15] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[16] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[17] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[18] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[19] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[20] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[21] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[22] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[23] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[24] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[25] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[26] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[27] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[28] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[29] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[30] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[31] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[32] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[33] >= 0.8: print("해당 "+filenames[cnt].split("/content/drive/My Drive/Colab Notebooks/MIX/bird")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")

    cnt += 1
    # print(i.argmax()) #얘가 레이블 [1. 0. 0.] 이런식으로 되어 있는 것을 숫자로 바꿔주는 것.
    # 즉 얘랑, 나중에 카테고리 데이터 불러와서 카테고리랑 비교를 해서 같으면 맞는거고, 아니면 틀린거로 취급하면 된다.
    # 이걸 한 것은 _4.py에.
